# Course Theme: Understanding Fastscape Fundementals and Applying Fastscape to Data

# Notebook 2: Fastscape Theory in the Basin

<img src="./02images/awaymountains.jpg" width="500">


### Away from the mountains and into the wild west of the basin...

- Notice many of these papers and theory were written since 2019!

# Objectives: 
1) Introduce the Sediment Model Equation for Landscape Evolution for basins;
2) discuss implementations in Fastscape 
3) Introduce Basin Steady State
4) Plot an example of Basin Steady-state
5) Discuss additional Parameters helpful for Basin Analysis

# 1) Theory: Applying the Sediment Equation to a Basin witihn Fastscaope

## 1.1) Sediment Model Equation (Erosion+Deposition): G

- Especially in areas of no tectonic uplift (the basin), sediments are deposited to form a piedmont and, ultimately, a sedimentary basin. To model this process, we must add a depositional term to the SPL equation. In FastScape, we have implemented an efficient version of the algorithm developed by Davy and Lague (2009) https://doi.org/10.1029/2008JF001146: $\frac{\partial h}{\partial t} = U - K_f A^mS^n + \frac{G}{A}q_s$ where q_s is the local sedimentary flux and G a dimensionless constant.

- The deposition parameter G can be added to account for the in-channel interaction with an active layer of sediment. This deposition term means that the channel can switch between detachment limited (incision control) vs transport limited (transport and deposition control). The equation tends towards detachment limited (G=0) to transport limited (G>0.4). 

-  Input: Laure Gurie (https://doi.org/10.1130/G46356.1) has done more work better constraining input values of G (range) within the modelling and real world. In the real world, G ∈ [0.4 − 1.2] with a median value of 0.7. Steep rivers and small catchments tend to be associated with low values of G and vice versa (Guerit et al., 2019).

- In Fastscape, use values under 1 within Fastscape. As G increases, your solution becomes more diffusive (erosion rate becoming more proportional to curvature rather than slope). 

- G is also influenced by n. A more linear relationship between slope and erosion would lead to higher values of G but in the same order of magnitude (Guerit et al.,2019). 

- G can also be split into a precipitation component as $Gv^-1$. This means that the precipitation is now inherent in the steady-state equations twice (once from K and once from G) emphasizing that precipitaiton plays a larger role (exponent of 1( from G) and m (from K) in the sediment equation compared to the basic equation. 


## 1.2) Implementation: Building a Basin in Fastscape


- Fastcsape: uses the algorithm as developed in [Yuan et al, 2019] (https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018JF004867) that is $O(n)$ and implicit and is calculated along the drainage stack. It requires, however, an iterative scheme that is rapidly converging but for $G<1$ only. 

- In the sediment model, we can differentiate between values of K and G for bedrock and for sediment.

- Use multiflow for more complex basin drainage patterns

### If you want to have a basin, you need to leave a portion of the model grid space that is not uplifting. 

Thus, we will now be building a raster grid for uplift/subsidence rather than inputing a single value. 

Using a grid space for uplift opens up a range of new possibilities (we won't have time to cover them all):
- You could impose a known/estimated subsidence decay at the foot of a mountain within a raster grid
- You could import a topography raster (DEM) and modify, use it to inform an uplift and subsidence grid rate based on prexisting mountain and basin areas
- You could use a flexure add on to estimate the subsidence in the areas not uplifting with the basin. 

In [141]:
#Summary on the board...(time allowing)

# 2) Theory: Useful Parameters to Describe/Control Basin Dynamics

## 2.1) Basin Steady-State

As described in Braun,2022 (https://doi.org/10.5194/esurf-10-301-2022) :
- Basin steady state is defined as when the change (over time) in the maximum surface elevation of the sedimentary system becomes constant (change in height over time of the fan apex) and the basin profile has reached its steady-state shape (change in slope over time).
- Basin steady state is the response time needed for the system shape to respond to change in its external forcings (incoming sediment flux or precipitation rate).
- Basin steady-state does not indicate that a basin is fillled (more on this latter) as with a certain amount of flux and accommodation space, the basin may never fill. 
- Equations for basin steady state are more complicated than for the orogenic area. Instead, there are three different equations that vary depending on basin length relative to cathcment area and erosion or deposition dominance (see equations 26-28 in Braun, 2022). 

<img src="./02images/BasinSteady_State.png" width="500"> (Braun, 2022)

- L is basin length, qs0 is flux from the upstream catchment area, w is basin (model) width, A0 is upstream area, v is precipitation, G is the depositional coefficient, Kf is the fluvial erodiblity, m and n are the stream power law exponents (see notebook 1).
- Notice that the equations differ depending on if erosion (eg:low G) or deposition (eg: high G) dominate the basin, or if the downstream length (L) of the basin is greater than or less than the length of the catchment (source). See Braun (2022) for details and full parameter definitions. 
- This is because confinement of the drainage system (ie: space to fill) controls the time needed to fill it. 

##### Precipitation: 
- Later will will seperate out a precipitation parameter within Fastscape. This gets applied to calculate the final Kf and G. Precipitation is thus, playing a larger role in contolling steady-state than in the basic equation (contributes to two parameters in the equation). 
- Lowering the precipitation will then decrease the erodiblilty (Kf) and increase the G (more diffusive or transport limited).
- Lower precipitation will mean that the model will take longer to reach steady-state due to the impact of precipitation on Kf. 

- example of the impact of precipitation on basin steady-state (apex height over time with a mountain catchment of 50 km and a basin of 200km with a subsidence exponential decay (alpha) of 5.)

<img src="./02images/PrecipitationImpactSteady-state.png" width="500">

## 2.2) Over-Fill (Bypass) or Under-Fill

Definition: The ratio of incoming sediment flux from the catchment to the basin accommodation space.
High F: flux is leaving the basin
Low F: All incoming flux is maintained within the basin.


Measuring the underfill and bypass is the ratio of flux to accommodation space. 

- At steady-state, basins can remain in underfill if the imposed accommodation space is much larger than the incoming flux. Thus, the basin may never evolve to be filled. 
- Non-steady-state conditions are often underfilled, but not necessarily. A filled basin will still take time to transition to a new constant height after a external perturbation.

$F = \frac{q_{s,i}}{(1- \gamma_p )L_B \int_0^{1}  \sigma (x') dx'} $

- Underfill and flat topographic solutions are often frequentend by inundation and local minima. Fastscape can simulate this, but the drainage dynamics are totally rerouted (for more information on the rerouting see Cordonnier et al. (2019) (https://doi.org/10.5194/esurf-7-549-2019)). Keep this in mind if you are trying to analyze drainage (or drainage influeced) solutions. 

- Bypass solutions eventually converge to a constant height no matter how much more flux you feed in or how much more you increase F. The F and incoming flux in overfill will control how long it takes to reach a steady-state height however. Underfill solutions (F<1), each increase in F and incoming flux will produce a different max height. 

- Between underfill and high bypass (ex. filled or just over-filled) the basin can still be impacted by local minima

<img src="./02images/ChannelMob_LocalMin.png" width="500">

- Figure shows increasing the initial subsidence rate (highest is purple) and basin accommodation causing a lower F (less sediment is leaving the basin as there is more accommodation space). 
- Low F show high local minima (big hole to fill) and control the channel mobility dynamics of the 3 purple solutions. 

## 2.3) Fan Extent/Transition (At steady-state)

Definition: The transition point from upstream catchment dominance to basin dominance.
Definition: Downstream break in slope where the steep alluvial fan transitions to a more gradual alluvial plain. 

$\beta=\frac{\nu_ML_M\alpha}{\nu_BL_B}$

-$v_m$ is precipitation in the mountain catchment, $L_m$ is the length of the mountain catchment, $\alpha$ is subsidence exponential decay, $v_B is precipitation in the basin, L_B is length of the basin

<img src="./02images/BetaCHangesAlpha5.png" width="400">

<img src="./02images/NormalizedTopo_BetaCHangesAlpha5.png" width="400">

- where beta is above alpha, the basin slope is concave and below convex. 
- where beta matches alpha, the profile is near linear. 
- higher beta is exerting a greater upstream control further (and beyond) the basin, thus no transition from fan to plain is observed (mega fan).
- Flux coming into the basin isn't changing, but deposition and erosion (stream power) is!

## 2.4) Basin Height (without subsidence and at steady-state)


- Without subsidence, q0, Kf (fluvial erodibility), G, w, Kd (hillslope diffusion) and A0 control the height of the depositional system at steady-state (Braun, 2022).
- Basin height will also be impacted by underlying subsidence and vary throughout the transient infilling phase

## 2.5) Additional

- defining the steady-state fan slope at the apex and toe is described in equation 20 (Braun, 2022).
- Flux coming out (leaving) the basin can be defined as: $qs_{out}=[\frac{\frac{dh}{dt}(\sigma)+(KA^mS^n)}{G}]A$
- Through the grain size fucntion we will also calculate flux by integrating the erosion and deposition rate through the drainage stack.

# 3) Theory: Grain Size 

## 3.1) Grain Size Theory and Model Iputs

#### Fedele and Paola (2007) Equation:

- The underlying approach for our integration of grain size into a landscape evolution model is the based on the Fedele and Paola (2007) (https://doi.org/10.1029/2005JF000409) self-similar down-system rate of grain size fining within depositional areas for gravel ($D(x^*) = \overline{D_0} + \phi_0  \frac{C_2}{C_1}  e^{-C_1y^*}-1$) and sand ($	D(x^*) = \overline{D_0}  e^{-C_3y^*}$).

- We will discuss other inputs later and focus on y* (dominant control on fining rate within this equation). 

#### y* calculation (in Fastscape)

- The following approach highlights a method to abandon the length scaling allowing the Fedele and Paola (2007) self-similar grain size model to be applied in 2D/3D within Fastscape. 

- Combing the Fedele and Paola (2007) equations ($y^*(x^*) = \int_0^{x^*} R^*(x^*) dx^*$ and $R^*= (1-\gamma_p)\frac{r}{qs}L$) and using the $x^*$ length scaling (x* = downstream distance (x)/total basin length(L)) produces the following equation: $y^*(x^*) = \int_0^{x^*} (1-\gamma_p) \frac{r(x^*)}{qs(x^*)}  L dx^*$

- In Fastscape, the deposition (negative)/erosion (positive) is the derivative of the flux. Thus r becomes: $r(x^*)= - \frac{dqs}{dx^*} \frac{1}{L}$.

- Substituting the Fastscape r(x*) into the combined Fedele and Paola (2007) equation ($y^*(x^*) = (1-\gamma_p)  \int_0^{x^*} -\frac{dqs\times L}{dx^* \times qs \times L}  dx^*$) we see that the length scalings cancel becoming: $y^*(x^*) = (1-\gamma_p)  \int_0^{x^*} -\frac{dqs}{ qs} $

- Thus by integrating the deposition (dqs) over the flux (qs) through the drainage network, we calculate a dimensionless $y^*(x^*)$ necessary for the grain size in 2D. 

- Y* is calculated in real time within Fastscape through the packages (in the folder with this notebook) GFast_Gravel and GFast_Sand. 
- GFast_Gravel and GFast_Sand are two new processes that use outputs from fastscape to calculate grain size fining based a a source distribution in real time. 

#### GRain size Inputs

Gravel Parameters and Inputs to the model are:
- See Fedele and Paola (2007)( https://doi.org/10.1029/2005JF000409 and Duller et al.(2010) (https://doi.org/10.1029/2009JF001495) for details. 
- D0 source grain size distribution mean or D50: Enter what you would like as the catchement source grain size. I like to normalize this D0 and the SD0 to a dimensionless variable where the D0 is 1 and S0 varies between~0.1-1. 
- S0 source grain size distribution standard deviation:
- C1:C1 is a coeffiecent derived from many river systems as the change in standard deviation ($S_D$) over the change in dimensionless downstream distance (x*)  ($C_1 = \frac{d S_D}{dx*}$).Fedele and Poala (2007) state 0.5 < C1 < 0.9, with an average value for C1 ~0.75 for gravels.
- Cv: is the coefficient of variation (downstream change in SD over downstream change in grain size distribution mean). Cv=C1/C2, Cv=SD0/D0, and $C_v$ ranges btw 0.7-0.9 (Fedele and Paola, 2007). Whittaker et al. (2011) highlights that a Cv of 0.8 is common in many river systems. 
- C2: Is the change in mean grain size (D) over the change in downstream distance ($C_1 = \frac{d D}{dx*}$). C2 can be derived from CV if C1, SD0, and D0 required inputs are known. 
- porosity:ranges between 0-1.


For Sand inputs are:
- D0 source grain size distribution mean or D50:
- S0 source grain size distribution standard deviation:
- C3: C3 (constant)is the coefficient of variation for sands described in Fedele and Paola (2007) and Duller et al. (2010). Usually ranges between 0.1-0.45. 0.3 is the average value for sands from Fedele and Poala (2007).
- porosity:ranges between 0-1.


## 3.2) Grain Size Application to Fastscape

In Fastscape:
- Always use the sediment model (https://fastscape.readthedocs.io/en/latest/models.html) and ideally multiflowrouter (https://fastscape.readthedocs.io/en/latest/_api_generated/fastscape.processes.MultipleFlowRouter.html#fastscape.processes.MultipleFlowRouter) process add ons in Fastscape when calcualting the grain size.

### GFast_Gravel.py, GFast_Sand.py, GSFast_Functions.py Files

- The GFast_Gravel and GFast_Sand packages need to be in your folder with your notebook along with the GSFast_functions.
- deposition/erosion is intergrated through the drainage stack to calculate the flux (see the GSFast_functions file for the code for this).
- Then y* is calculated (deposition/flux) intergrated through the drainage stack where a weighted mean (based on discharge/drainage area) is applied when channels merge. This is based on Harries et al (2019) work (doi:10.1111/bre.12349) that states that when channels merge, the dominant discharge channel tends to dominate the grain size distribution. 1-the porosity is also multiplied to the Y* calculation. 
- The source distribution is selected from what was last deposited at the bed wherever the channel initiates. 


### 3.3) GSFast Outputs

 GFast_Gravel and GFast_Sand can output:
- grain size fining at a given time step (gsize__DMean). ONly what was deposited at one surface.
- an updating grain size surface over time (gsize__DTIME). Updates what is on the surface. 
- the source grain size (gsize__D0Source) (using what is at the bed whereever the channel initiates and propogated downsteam).
- The model age of the deposit (gsize__Age)
- The sediment flux derived through integrating through the drainage stack (gsize__EFlux). 

Notes:
- Keep in mind that the grain size is extremely sensitive to changes in flux, deposition, channel dynamics and local minima. 

<img src="./02images/grainsize.png" width="400">
- Example difference between Dmean (orange) and DTime(green) outputs for a given time step where DMean is what was specifically deposited at that time step and DTIME is the surface with everything that had been deposited beforehand. 

# 4) Extra basin functions

### DominantChannelDynamics.py File

Channel Mobility
- Functions AvulsionMainChannel was designed for channels flowing in one direction from a orogenic source either in the x or y direction. 
- In all the examples above, downstream from the orogenic front is along the x axis. 
- Channels then migrate/avulse from their pathways along the y axis. 
- AvulsionMainChannel takes 1)Binary: an empty (filled with zeros) array of the dimensions (eg (non-batch input with 3 dimensions): time, y, and x) of the Fastscape drainge output.
    2) temp: the location of the dominant drainage pathway for each time step. 
- AvulsionMainChannel can take batch inputs, but they need to be stacked as one batch input (eg: 4 dimensions). 
- AvulsionMainChannel outputs a binary grid of where in the x and y the position of the channel changed between time steps (a mobility event).
- In post processing, this can be summarized and divided by the time steps-1 in quesiton to derive a mobility frequency. 
- Examples will follow in notebook 3. 


Local Minima:
- You could easily calculate where the topography upsteam minus downstream is negative or 0 to calculate the location of local minima at a given time step. 

- Functions find_slopes_withMinima (h, stack, rec,nrec) takes the drainage stack, recievers, and number of recieves, and topography elevation as inputs (1D arrays- need to turn topography into a 1D array) to 
calculate the slope between nodes. Where local minima occur (slopes less than or equal to zero), the slope is set to zero. In post-processing you can than calculate where the slope is zero as the local minima locations in the stack. 



### Stratigaphy.py File

- plotStratigraphy takes 1) XorY_StratiOverTime (time and either x or y dimensions): strati__elevation selected for only the basin area and either averaged or selected for one across (y)/down(x) basin distance 
    2) XorY_GrainSizeOverTime (time and either x or y dimensions) the grain size or erosion rate or other desired variable that will be used to fill the stratigraphy. This also needs to be selected or averaged for one x/y distance. 
- stratigraphy as it is written assumes that channels are draining either in the x or y direction (mountain along one axis) and stratigraphy is generated along one axis.     
- plotStratigraphy averages the nearby nodes (grain size or erosion rate or other desired value passed) to fill a given cell of stratigraphy.
-plotStraigraphy2 does not average the nearest nodes and takes the first closest value to fill the stratigraphy. 
-Example shown in notebook 3.

<img src="./02images/BasinF_depo_channelMob.png" width="500">
- Example stratigraphy (mountain to the left and draining to a sink (right) filled with a calculated F value, channel mobility, and erosion/deposition rate)
- This shows the development of a foreland basin over time with flexure